In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,31852
2,Huelva,Confirmados PDIA 14 días,1533
3,Huelva,Tasa PDIA 14 días,"298,73141454099033"
4,Huelva,Confirmados PDIA 7 días,722
5,Huelva,Total Confirmados,32032
6,Huelva,Curados,29381
7,Huelva,Fallecidos,384


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  31852.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8546.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 464 personas en los últimos 7 días 

Un positivo PCR cada 237 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,31852.0,722.0,1533.0,513170.0,140.694117,298.731415,101.0
Huelva-Costa,18333.0,408.0,836.0,289548.0,140.909279,288.725876,64.0
Huelva (capital),8546.0,310.0,607.0,143837.0,215.521736,422.005465,55.0
Condado-Campiña,10352.0,238.0,550.0,156231.0,152.338524,352.042808,30.0
Bollullos Par del Condado,999.0,26.0,51.0,14387.0,180.718704,354.486689,7.0
Sierra de Huelva-Andévalo Central,2817.0,72.0,141.0,67391.0,106.839192,209.226751,6.0
Moguer,1309.0,32.0,89.0,21867.0,146.339233,407.005991,5.0
Bonares,279.0,8.0,15.0,6060.0,132.013201,247.524752,5.0
San Juan del Puerto,524.0,26.0,47.0,9411.0,276.272447,499.415578,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,177.0,9.0,14.0,2211.0,407.055631,633.197648,0.0
Niebla,239.0,5.0,25.0,4116.0,121.477162,607.385811,0.0
Valverde del Camino,599.0,37.0,67.0,12750.0,290.196078,525.490196,2.0
Palos de la Frontera,681.0,38.0,60.0,11742.0,323.624595,510.986203,2.0
San Juan del Puerto,524.0,26.0,47.0,9411.0,276.272447,499.415578,4.0
Almonte,1843.0,36.0,104.0,24507.0,146.896805,424.368548,1.0
Huelva (capital),8546.0,310.0,607.0,143837.0,215.521736,422.005465,55.0
Palma del Condado (La),1161.0,15.0,45.0,10801.0,138.876030,416.628090,1.0
Moguer,1309.0,32.0,89.0,21867.0,146.339233,407.005991,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Isla Cristina,2150.0,2.0,21.0,21393.0,9.348852,98.162950,0.0,0.095238
Calañas,191.0,1.0,6.0,2768.0,36.127168,216.763006,0.0,0.166667
Nerva,178.0,1.0,5.0,5169.0,19.346102,96.730509,0.0,0.200000
Niebla,239.0,5.0,25.0,4116.0,121.477162,607.385811,0.0,0.200000
Rociana del Condado,558.0,4.0,16.0,7939.0,50.384179,201.536717,1.0,0.250000
Lepe,2135.0,8.0,26.0,27880.0,28.694405,93.256815,1.0,0.307692
Ayamonte,1322.0,4.0,13.0,21104.0,18.953753,61.599697,2.0,0.307692
Lucena del Puerto,255.0,3.0,9.0,3261.0,91.996320,275.988960,1.0,0.333333
Palma del Condado (La),1161.0,15.0,45.0,10801.0,138.876030,416.628090,1.0,0.333333
